In [1]:
import hyperspy.api as hys
import numpy as np
import tkinter.filedialog as tkf

In [2]:
%matplotlib qt

In [ ]:
%matplotlib inline

In [3]:
hl_tmp = hys.load(tkf.askopenfilename(), signal_type = "EELS")
print(hl_tmp)

<EELSSpectrum, title: spectrum_A_metadata, dimensions: (|2048)>


In [4]:
hl = hl_tmp.deepcopy()

In [5]:
hl.crop_signal1D()

In [6]:
hl.add_elements(("Co", ))

In [7]:
m = hl.create_model(auto_background=False, GOS="Hartree-Slater")
print(m.components)

   # |      Attribute Name |      Component Name |      Component Type
---- | ------------------- | ------------------- | -------------------
   0 |               Co_L3 |               Co_L3 |          EELSCLEdge
   1 |               Co_L2 |               Co_L2 |          EELSCLEdge


In [8]:
m.multifit(kind="smart")

In [13]:
m.plot()
m.enable_adjust_position()

In [10]:
print(m.components.Co_L3.onset_energy.value)
print(m.components.Co_L2.onset_energy.value)
m.components.Co_L3.onset_energy.assign_current_value_to_all()

779.4468217804139
794.4468217804139


In [14]:
roi = hys.roi.SpanROI(left=750, right=800)
m.plot()
roi.add_widget(m, axes=["Energy loss"])

In [15]:
nor_line = np.mean(m.signal1D.isig[roi].data)
nor_ratio = nor_line / np.mean(m.as_signal().isig[roi].data)
print(nor_ratio)
continuum = m.as_signal() * nor_ratio
print(continuum)

0.8000540197237006
<EELSSpectrum, title: spectrum_A_metadata from fitted model, dimensions: (|362)>


In [16]:
hys.plot.plot_spectra([m.signal1D, continuum])

In [17]:
roi = hys.roi.SpanROI(left=750, right=800)
m.signal1D.plot()
roi.add_widget(m.signal1D, axes=["Energy loss"])

In [18]:
l3_raw = m.signal1D.isig[roi].integrate1D(axis="Energy loss")
print(l3_raw.data)
l3_hs = continuum.isig[roi].integrate1D(axis="Energy loss")
print(l3_hs.data)

[106417.945]
[32620.31715786]


In [19]:
roi = hys.roi.SpanROI(left=750, right=800)
m.signal1D.plot()
roi.add_widget(m.signal1D, axes=["Energy loss"])

In [20]:
l2_raw = m.signal1D.isig[roi].integrate1D(axis="Energy loss")
print(l2_raw.data)
l2_hs = continuum.isig[roi].integrate1D(axis="Energy loss")
print(l2_hs.data)

[66108.49]
[49192.45938833]


In [22]:
wl_ratio = (l3_raw - l3_hs) / (l2_raw - l2_hs)
print(wl_ratio.data)

[4.36258483]
